<a id="toc"></a>
# Table of Contents
## [Python for Data Analysis (pandas)](#pandas)
## [Querying Dataframes](#querying)
### [Query Challenge](#query_challenge)
## [Reshaping Data](#reshaping)
## [Aggregating Data](#aggregates)
## [Merging Datasets](#merging)
## [Ordering Data](#ordering)
## [Exporting Data](#export)

<a id="pandas"></a>
## Pandas
[Back to Table of Contents](#toc)

Pandas is a _library_ which allows us to do some powerful operations with table-like data. We can query datasets with a high degree of granularity, merge them together, sort, and aggregate them.

I highly suggest you take a look and read through the documentation on it when you can! **[It is available here.](https://pandas.pydata.org/pandas-docs/stable/index.html)**

Combined with this lesson are 4 Excel spreadsheets that we will be combining together to form various queries.

- `COMPANIES.xlsx`
- `EMPLOYEES.xlsx`
- `FAKE_DATA_BUILD.xlsx` <- This is our main table
- `ISO_COUNTRY_LOOKUPS.xlsx`

To start with, let's read a spreadsheet and see the output.

Remember, if we want to use a _library_ we have to **import** it.

In [68]:
import pandas as pd # we can create a temporary name using the "as" keyword here which can make it shorter

pd.set_option('display.max_rows', 1000) # We can use this variable to decide how many rows we want to see
pd.set_option('display.max_columns', 500) # We can use this variable to decide how many columns we want to see

df_employees = pd.read_excel('EMPLOYEES.xlsx')
df_employees # This will display the table

,EMPLOYEE_ID,FNAME,LNAME,DOB,OFFICE_CTRY_CODE
0,QZE4337,Domingos,Bello,1964-09-13,BL
1,QVJ3467,Kira,Gomes,1990-06-12,MF
2,NCD9017,Modesto,Fernández,1974-06-28,MQ
3,DHI8102,Samuel,Escárcega,1996-09-08,MQ
4,VDS7914,Raya,Bustillo,1983-11-30,MQ
5,EWJ3178,Lavrentiy,Gallo,1982-05-13,GP
6,KFW4978,Ofelia,Rojas,1995-08-10,GP
7,ZJQ3093,Estela,Amador,1979-07-17,GP
8,YVK2375,Leonti,Fernández,1996-11-22,GP
9,OGQ7027,Lourenço,Fernández,1971-01-18,GP


Above, I've named the table "df_employees". Since this table is going to hold the employee information, I've called it "employees" but I've added the prefix "df_", why?

In pandas, table objects are stored as what are called "dataframes" - I'm using "df" as shorthand for that to let me know what kind of _object_ I'm working with.

Let's read our other sheets into dataframe objects so we can begin using them.

In [69]:
df_main = pd.read_excel("FAKE_DATA_BUILD.xlsx", sheet_name="Main")
df_main_definitions = pd.read_excel("FAKE_DATA_BUILD.xlsx", sheet_name="VariableDefinitions")
df_companies = pd.read_excel("COMPANIES.xlsx")
df_countries = pd.read_excel("ISO_COUNTRY_LOOKUPS.xlsx")

In [71]:
# write each dataframe name here individually and see what it outputs when you run this

df_main_definitions

,DATE,Date that the transaction originated
0,ORGN_CTRY_CODE,The ISO 3166-2 code of the exporting country
1,DEST_CTRY_CODE,The ISO 3166-2 code of the importing country
2,PKG_HT,Height of the package
3,PKG_LT,Length of the package
4,PKG_WDT,Width of the package
5,TRANSPO_TYPE,Type of transportation
6,CUSTOMS_TYPE,Type of customs declaration
7,DECLARED_VALUE_USD,Declared value of the shipment
8,SOLUTION_TYPE,Type of solution (FF or SP)
9,SALES_CTRY_CODE,The ISO 3166-2 code of the owner sales team


Next, we can learn a little bit about our data using a few small tools:

- `len()` <- tells us how many rows are in the dataframe.
- `dataframe.info()` <- gives us basic information about the dataframe.
- `dataframe.columns.values` <- Gives us the headers of the dataframe (useful later).

In [72]:
print('Number of rows in df_main: ' + str(len(df_main)))
print('Number of rows in df_employees: ' + str(len(df_employees)))
print('Number of rows in df_companies: ' + str(len(df_companies)))
print('Number of rows in df_countries: ' + str(len(df_countries)))

Number of rows in df_main: 44232
Number of rows in df_employees: 486
Number of rows in df_companies: 1950
Number of rows in df_countries: 249


Let's take a look at df_main and see what we can learn about our data.

In [73]:
df_main.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44232 entries, 0 to 44231
Data columns (total 17 columns):
DATETIME              44232 non-null datetime64[ns]
ORGN_CTRY_CODE        44232 non-null object
DEST_CTRY_CODE        44232 non-null object
PKG_LENGTH            44232 non-null int64
PKG_HEIGHT            44232 non-null int64
PKG_WIDTH             44232 non-null int64
TRANSPO_TYPE          44232 non-null object
CUSTOMS_TYPE          44232 non-null object
DECLARED_VALUE_USD    44232 non-null int64
SOLUTION_TYPE         44232 non-null object
SALES_CTRY_CODE       44232 non-null object
EMPLOYEE_ID           44232 non-null object
COMPANY_ID            44232 non-null object
INVOICE_NUM           44232 non-null object
INVOICE_REV           44232 non-null float64
INVOICE_PAID          44232 non-null bool
DATE_PAID             35451 non-null datetime64[ns]
dtypes: bool(1), datetime64[ns](2), float64(1), int64(4), object(9)
memory usage: 3.9+ MB


In [74]:
df_main.columns.values

array(['DATETIME', 'ORGN_CTRY_CODE', 'DEST_CTRY_CODE', 'PKG_LENGTH',
       'PKG_HEIGHT', 'PKG_WIDTH', 'TRANSPO_TYPE', 'CUSTOMS_TYPE',
       'DECLARED_VALUE_USD', 'SOLUTION_TYPE', 'SALES_CTRY_CODE',
       'EMPLOYEE_ID', 'COMPANY_ID', 'INVOICE_NUM', 'INVOICE_REV',
       'INVOICE_PAID', 'DATE_PAID'], dtype=object)

Although looking at the data is useful, we need to learn a little more about what each column contains. In this dataset, we have a "VariableDefinitions" sheet what we pulled into df_main_definitions. Let's take a look.

In [ ]:
df_main_definitions

<a id="querying"></a>
## Querying Dataframes
[Back to Table of Contents](#toc)

Reading data is useful but without the ability to ask a dataset questions, it doesn't really give us much more over Excel. This is where the strength of pandas starts to show.

The _syntax_ for querying is fairly simple but can get complex based upon the query itself.

For our purposes, I'll describe it this way:

The statement: `df_main[df_main["ORGN_CTRY_CODE"] == "BR"]` can be read as "within the dataframe df_main tell me where 'ORGN_CTRY_CODE' equals 'BR'." Or, more simply: all data for exports from Brazile.

We can also utilize multiple conditions like so:

`df_main[(df_main["ORGN_CTRY_CODE"] == "BR") & (df_main["DEST_CTRY_CODE"] == "US")]` - this can be read as "within the dataframe df_main tell me where 'ORGN_CTRY_CODE' equals 'BR' **_and_** 'DEST_CTRY_CODE' equals 'US'. Or, more simply, all data for packages exported from Brazil to the US.


In [75]:
df_main[df_main["ORGN_CTRY_CODE"] == "BR"]

,DATETIME,ORGN_CTRY_CODE,DEST_CTRY_CODE,PKG_LENGTH,PKG_HEIGHT,PKG_WIDTH,TRANSPO_TYPE,CUSTOMS_TYPE,DECLARED_VALUE_USD,SOLUTION_TYPE,SALES_CTRY_CODE,EMPLOYEE_ID,COMPANY_ID,INVOICE_NUM,INVOICE_REV,INVOICE_PAID,DATE_PAID
6,2017-07-09 03:41:37.064,BR,SV,48,58,40,Air,INFORMAL,1159,FF,BR,LML1551,69.596.270/0001-03,A429251551026682169,418.00,True,2017-07-14 03:41:37.064
16,2019-06-14 22:58:12.588,BR,US,87,18,11,Ocean,INFORMAL,1370,SP,US,OWM9589,66.848.689/0001-17,O436319589018583112,24.60,False,NaT
20,2016-03-19 21:33:03.879,BR,PY,77,3,12,Air,FORMAL,6665,SP,BR,NEI7590,35.217.629/0001-72,A424497590016682107,159.50,True,2016-08-30 21:33:03.879
55,2014-08-06 09:36:49.684,BR,MX,28,31,10,Air,INFORMAL,1155,SP,BR,MRS8259,19.635.017/0001-43,A418578259016682161,22.00,False,NaT
107,2018-06-16 10:45:58.878,BR,US,19,10,33,Air,INFORMAL,2549,SP,US,YLR3829,10.961.832/0001-28,A432673829018583138,24.60,True,2018-10-06 10:45:58.878
109,2017-11-22 06:27:15.779,BR,MQ,48,46,40,Air,INFORMAL,2233,FF,MQ,YTN1881,61.463.169/0001-72,A430611881027781103,382.42,True,2018-05-02 06:27:15.779
118,2017-11-22 07:38:46.070,BR,GT,102,10,12,Air,INFORMAL,1409,SP,BR,WPA4184,74.896.058/0001-00,A430614184016682138,22.00,True,2018-05-12 07:38:46.070
135,2014-03-10 20:02:14.410,BR,US,79,19,13,Air,FORMAL,3133,SP,BR,ELM3711,93.618.981/0001-16,A417093711016682119,159.50,True,2014-07-22 20:02:14.410
165,2018-10-17 01:37:31.021,BR,CA,46,14,9,Rail,INFORMAL,1042,SP,BR,ASH6956,00.708.772/0001-82,R433906956016682193,22.00,False,NaT
188,2016-07-19 13:13:27.549,BR,US,25,12,3,Truck,INFORMAL,1977,SP,BR,QVC1672,06.294.680/0001-80,T425711672016682188,22.00,True,2016-12-09 13:13:27.549


In [76]:
df_main[(df_main["ORGN_CTRY_CODE"] == "BR") & (df_main["DEST_CTRY_CODE"] == "US")]

,DATETIME,ORGN_CTRY_CODE,DEST_CTRY_CODE,PKG_LENGTH,PKG_HEIGHT,PKG_WIDTH,TRANSPO_TYPE,CUSTOMS_TYPE,DECLARED_VALUE_USD,SOLUTION_TYPE,SALES_CTRY_CODE,EMPLOYEE_ID,COMPANY_ID,INVOICE_NUM,INVOICE_REV,INVOICE_PAID,DATE_PAID
16,2019-06-14 22:58:12.588,BR,US,87,18,11,Ocean,INFORMAL,1370,SP,US,OWM9589,66.848.689/0001-17,O436319589018583112,24.60,False,NaT
107,2018-06-16 10:45:58.878,BR,US,19,10,33,Air,INFORMAL,2549,SP,US,YLR3829,10.961.832/0001-28,A432673829018583138,24.60,True,2018-10-06 10:45:58.878
135,2014-03-10 20:02:14.410,BR,US,79,19,13,Air,FORMAL,3133,SP,BR,ELM3711,93.618.981/0001-16,A417093711016682119,159.50,True,2014-07-22 20:02:14.410
188,2016-07-19 13:13:27.549,BR,US,25,12,3,Truck,INFORMAL,1977,SP,BR,QVC1672,06.294.680/0001-80,T425711672016682188,22.00,True,2016-12-09 13:13:27.549
192,2018-06-03 12:36:14.667,BR,US,13,2,15,Air,FORMAL,4451,SP,BR,IQL9168,65.730.134/0001-03,A432559168016682191,159.50,True,2018-10-03 12:36:14.667
211,2013-07-06 09:29:07.311,BR,US,15,28,22,Air,INFORMAL,827,SP,BR,KBH6717,66.269.037/0001-28,A414616717016682126,22.00,True,2013-07-27 09:29:07.311
245,2018-11-28 19:16:54.591,BR,US,16,19,10,Air,INFORMAL,1763,SP,BR,VZA6337,64.983.298/0001-80,A434336337016682131,22.00,False,NaT
256,2017-11-25 02:34:44.033,BR,US,6,5,2,Air,INFORMAL,575,SP,BR,FNT2506,27.242.147/0001-44,A430642506016682192,22.00,True,2018-05-22 02:34:44.033
278,2018-03-05 17:41:27.869,BR,US,29,5,24,Air,INFORMAL,1479,SP,BR,QOG8189,97.620.082/0001-09,A431658189016682186,22.00,True,2018-06-30 17:41:27.869
284,2016-10-16 18:32:50.702,BR,US,107,14,11,Air,INFORMAL,2163,SP,BR,LAF3668,99.286.187/0001-07,A426603668016682199,22.00,True,2016-11-22 18:32:50.702


<a id="query_challenge"></a>
### Query Challenge
[Back to Table of Contents](#toc)

Write the following queries:

- Get information about packages _from_ Mexico that were of transportation type "Truck"
- Get information about packages from Argentina whose invoices have _NOT_ been paid.
- Get information about packages from the US for 2017
    - Query dates like this: `(df_main["DATETIME"] >= '01-01-2017') & (df_main["DATETIME"] < '01-01-2018')`
- Get information about packages for all countries whose "SOLUTION_TYPE" was "FF" that have "INVOICE_REV" of greater than 500

In [79]:
df_main[(df_main["SOLUTION_TYPE"] == "FF") & (df_main["INVOICE_REV"] > 500)]

,DATETIME,ORGN_CTRY_CODE,DEST_CTRY_CODE,PKG_LENGTH,PKG_HEIGHT,PKG_WIDTH,TRANSPO_TYPE,CUSTOMS_TYPE,DECLARED_VALUE_USD,SOLUTION_TYPE,SALES_CTRY_CODE,EMPLOYEE_ID,COMPANY_ID,INVOICE_NUM,INVOICE_REV,INVOICE_PAID,DATE_PAID
12,2014-06-09 13:00:30.450,PE,US,48,55,40,Air,FORMAL,8126,FF,PE,MFR3102,937649-RWN,A418003102028069112,521.91,True,2014-11-22 13:00:30.450
24,2018-05-14 18:40:06.896,UY,BO,48,64,40,Air,FORMAL,9540,FF,UY,FNT2506,201321-QLC,A432352506028589177,509.83,False,NaT
31,2018-11-06 03:30:01.926,US,BR,48,49,40,Air,FORMAL,3671,FF,US,SOA5958,48-9578123,A434105958028583188,621.15,False,NaT
37,2018-10-11 16:08:19.838,UY,GT,48,63,40,Truck,FORMAL,7219,FF,UY,TUH1233,815913-ENI,T433851233028589126,509.83,False,NaT
38,2019-01-05 11:12:57.459,US,CA,48,68,40,Air,FORMAL,8022,FF,CA,FOG3270,98-2769108,A434703270026765147,565.60,False,NaT
65,2013-02-04 13:16:47.418,MX,GT,48,55,40,Truck,FORMAL,4871,FF,MX,GVG1704,718723-YUV,T413101704027788180,560.55,True,2013-06-06 13:16:47.418
72,2018-08-09 09:29:27.896,PR,MX,48,46,40,Truck,FORMAL,6569,FF,PR,NAK1050,605871-MTK,T433211050028082128,510.64,True,2018-09-22 09:29:27.896
100,2017-10-04 06:24:09.662,GT,MF,48,67,40,Air,FORMAL,4851,FF,GT,FTL2487,101395-IWS,A430122487027184192,519.49,True,2017-10-23 06:24:09.662
113,2014-09-03 18:47:40.038,US,BR,48,52,40,Air,FORMAL,8737,FF,US,BYH5668,80-3572471,A418865668028583102,621.15,True,2014-10-08 18:47:40.038
126,2013-04-29 21:04:17.983,HN,PE,48,62,40,Air,FORMAL,4651,FF,PE,EVT7333,108974-NYE,A413947333028069178,521.91,True,2013-07-17 21:04:17.983


<a id="reshaping"></a>
## Reshaping Data
[Back to Table of Contents](#toc)

Remember viewing the output of `dataframe.columns.values`? Let's use that to slim down the actual data we want for display.

We can get a subset of columns of a dataframe like this:

- `df_new` has headers `["col1", "col2", "col3", "col4"]`.
- We get a subset of these columns by creating a list of the columns we want and feeding it to the dataframe.
    - `df_new_subset = df_new[["col2", "col4"]]`

Let's get the headers of `df_main` again and take a look to see which ones we want - we'll convert them to a list as well.

In [80]:
hdrs_main = list(df_main.columns.values)
hdrs_main

['DATETIME',
 'ORGN_CTRY_CODE',
 'DEST_CTRY_CODE',
 'PKG_LENGTH',
 'PKG_HEIGHT',
 'PKG_WIDTH',
 'TRANSPO_TYPE',
 'CUSTOMS_TYPE',
 'DECLARED_VALUE_USD',
 'SOLUTION_TYPE',
 'SALES_CTRY_CODE',
 'EMPLOYEE_ID',
 'COMPANY_ID',
 'INVOICE_NUM',
 'INVOICE_REV',
 'INVOICE_PAID',
 'DATE_PAID']

Let's say our end goal is to get the total revenue for each employee. What columns would we need to do that? We'd probably need the employee ID and the invoice revenue - two columns "EMPLOYEE_ID" and "INVOICE_REV".

In [81]:
df_employee_revenue = df_main[["EMPLOYEE_ID", "INVOICE_REV"]]
df_employee_revenue

,EMPLOYEE_ID,INVOICE_REV
0,FOG3270,24.60
1,VWH4794,147.08
2,CDV8654,146.39
3,ONG7753,20.67
4,OTZ5266,20.45
5,OJC3762,22.40
6,LML1551,418.00
7,FAC1186,20.70
8,QZQ5495,20.64
9,CNM2808,24.60


<a id="aggregates"></a>
## Aggregating Data
[Back to Table of Contents](#toc)

It's great that we can get the columns that we need but there are two problems.
1. This is each individual transaction, it doesn't help us see the _total_ revenue for each employee.
2. We don't know who any of these employee IDs refer to (more on this later).

First, we need to _aggregate_ the data by summing the revenue for each employee.

The syntax is as follows:

`df_employee_revenue.groupby(["EMPLOYEE_ID"])["INVOICE_REV"].sum()`

We can also get the _average_ by using a different function:

`df_employee_revenue.groupby(["EMPLOYEE_ID"])["INVOICE_REV"].mean()`

Let's try it out.

In [82]:
df_tot_emp_rev = df_employee_revenue.groupby(["EMPLOYEE_ID"], as_index=False)["INVOICE_REV"].sum()
df_tot_emp_rev

,EMPLOYEE_ID,INVOICE_REV
0,AFG2372,6769.81
1,AIU2281,9696.23
2,AKN2433,7802.30
3,AKU3205,9874.56
4,ANI6151,8690.72
5,ANP8287,7163.46
6,AQN8786,9616.06
7,ARW9403,8990.35
8,ASH6956,10535.80
9,AYJ4798,9796.45


<a id="merging"></a>
## Merging Datasets
[Back to Table of Contents](#toc)

Aggregating data is useful but it doesn't solve the 2nd problem we mentioned earlier: we don't know who these people are. Luckily, we have another spreadsheet which has all of the employee information we need to solve this problem.

The way that we handle situations like this is to use pandas' _merge_ functionality.

Here's the syntax:

`df1.merge(df2, left_on='lkey', right_on='rkey')`

- **df1** is the dataset you want to merge outside data onto.
- **df2** is the dataset you want to merge onto df1.
- **left_on** is the name of the column in df1 you want df2 to merge onto.
- **right_on** is the name of the column in df2 you want to connect to the column in df1.
- If you don't put anything for the keys, it will attempt to find a matching pair of keys.

[Additional information can be found here.](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html)

Let's merge df_employees onto df_employee_revenue.

In [83]:
# Let's take a look at df_employees again to verify the columns and data
df_employees

,EMPLOYEE_ID,FNAME,LNAME,DOB,OFFICE_CTRY_CODE
0,QZE4337,Domingos,Bello,1964-09-13,BL
1,QVJ3467,Kira,Gomes,1990-06-12,MF
2,NCD9017,Modesto,Fernández,1974-06-28,MQ
3,DHI8102,Samuel,Escárcega,1996-09-08,MQ
4,VDS7914,Raya,Bustillo,1983-11-30,MQ
5,EWJ3178,Lavrentiy,Gallo,1982-05-13,GP
6,KFW4978,Ofelia,Rojas,1995-08-10,GP
7,ZJQ3093,Estela,Amador,1979-07-17,GP
8,YVK2375,Leonti,Fernández,1996-11-22,GP
9,OGQ7027,Lourenço,Fernández,1971-01-18,GP


In [84]:
df_tot_emp_rev

,EMPLOYEE_ID,INVOICE_REV
0,AFG2372,6769.81
1,AIU2281,9696.23
2,AKN2433,7802.30
3,AKU3205,9874.56
4,ANI6151,8690.72
5,ANP8287,7163.46
6,AQN8786,9616.06
7,ARW9403,8990.35
8,ASH6956,10535.80
9,AYJ4798,9796.45


In [85]:
# Let's merge
df_employee_rev_info = df_tot_emp_rev.merge(df_employees, left_on="EMPLOYEE_ID", right_on="EMPLOYEE_ID")

df_employee_rev_info

,EMPLOYEE_ID,INVOICE_REV,FNAME,LNAME,DOB,OFFICE_CTRY_CODE
0,AFG2372,6769.81,Lalo,Shwetz,1990-06-17,MX
1,AIU2281,9696.23,Quim,De León,1995-04-16,MX
2,AKN2433,7802.30,Gustavo,Rocha,1994-10-20,AR
3,AKU3205,9874.56,Benigno,Del Olmo,1981-08-06,AR
4,ANI6151,8690.72,Alyona,Herrero,1993-07-07,PY
5,ANP8287,7163.46,Timur,Fernández,1992-11-19,BR
6,AQN8786,9616.06,Lavrentiy,Araujo,1977-04-11,CA
7,ARW9403,8990.35,Estela,Petrov,1992-12-27,US
8,ASH6956,10535.80,Ofelia,Amador,1977-02-06,CA
9,AYJ4798,9796.45,Hermínio,Antúnez,1971-12-21,BR


<a id="ordering"></a>
## Ordering Data
[Back to Table of Contents](#toc)

Now that we've aggregated and merged out data, let's order our data and find the top 10 employees in our company! [More information is available here.](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html)

The syntax for sorting values is as follows:

`df.sort_values(by=['col1'])`

The column we want to sort by is "INVOICE_REV".

In [88]:
df_employee_rev_info = df_employee_rev_info.sort_values(by=["INVOICE_REV"], ascending=False)
df_employee_rev_info

,EMPLOYEE_ID,INVOICE_REV,FNAME,LNAME,DOB,OFFICE_CTRY_CODE
121,GGJ5606,15280.57,Sofía,Krupin,1965-03-13,US
235,MMM3364,14361.02,Fúlvio,Antúnez,1997-05-26,BR
15,BFJ9943,14101.47,Prokhor,Rojas,1973-08-23,US
71,ECC4877,14086.48,Andrea,Maksimov,1978-12-22,GT
70,EBF8874,14081.21,Prokhor,Krupin,1977-10-02,PY
186,KBH6717,13880.76,Estela,Matos,1966-09-07,VE
123,GID8381,13853.07,Eliana,Castilla,1982-06-05,US
168,IZK5191,13787.97,Cebrián,Echevarría,1984-07-06,CA
229,MCE9689,13615.09,Alex,Macías,1976-11-24,CA
410,VWH4794,13476.56,Cecilio,Espina,1990-01-07,MX


Let's get the top 10.

In [89]:
df_top_10 = df_employee_rev_info.head(10)
df_top_10

,EMPLOYEE_ID,INVOICE_REV,FNAME,LNAME,DOB,OFFICE_CTRY_CODE
121,GGJ5606,15280.57,Sofía,Krupin,1965-03-13,US
235,MMM3364,14361.02,Fúlvio,Antúnez,1997-05-26,BR
15,BFJ9943,14101.47,Prokhor,Rojas,1973-08-23,US
71,ECC4877,14086.48,Andrea,Maksimov,1978-12-22,GT
70,EBF8874,14081.21,Prokhor,Krupin,1977-10-02,PY
186,KBH6717,13880.76,Estela,Matos,1966-09-07,VE
123,GID8381,13853.07,Eliana,Castilla,1982-06-05,US
168,IZK5191,13787.97,Cebrián,Echevarría,1984-07-06,CA
229,MCE9689,13615.09,Alex,Macías,1976-11-24,CA
410,VWH4794,13476.56,Cecilio,Espina,1990-01-07,MX


<a id="exporting"></a>
## Exporting Data
[Back to Table of Contents](#toc)

Now that we've created our report, let's export it into Excel!

In [90]:
df_top_10.to_excel("our_cool_employee_report.xlsx", index=False)